In [2]:
from googleapiclient.discovery import build
import pandas as pd
from google.colab import files, drive,userdata
import getpass
from IPython.display import display, Markdown
from urllib.parse import urlparse, parse_qs


In [3]:
api_key = userdata.get('YT_API_KEY')

In [4]:
#get youtube video id from a link as "video id"

def get_video_id(url):
  """
  Extracts the video ID from a YouTube URL.

  Args:
    url: The YouTube URL.

  Returns:
    The video ID, or None if the URL is invalid.
  """
  parsed_url = urlparse(url)
  query_params = parse_qs(parsed_url.query)
  if 'v' in query_params:
    return query_params['v'][0]
  else:
    return None

video_url = input('Please enter the YouTube video link: ')
video_id = get_video_id(video_url)

if video_id:
  print("Video ID:", video_id)
else:
  print("Invalid YouTube URL.")
video_ids = [video_id]

Please enter the YouTube video link: https://www.youtube.com/watch?v=yBltrN9-uaQ
Video ID: yBltrN9-uaQ


In [5]:
youtube = build('youtube', 'v3', developerKey=api_key)


In [6]:


# Function to get replies for a specific comment
def get_replies(youtube, parent_id, video_id):  # Added video_id as an argument
    replies = []
    next_page_token = None

    while True:
        reply_request = youtube.comments().list(
            part="snippet",
            parentId=parent_id,
            textFormat="plainText",
            maxResults=100,
            pageToken=next_page_token
        )
        reply_response = reply_request.execute()

        for item in reply_response['items']:
            comment = item['snippet']
            replies.append({
                'Timestamp': comment['publishedAt'],
                'Username': comment['authorDisplayName'],
                'VideoID': video_id,
                'Comment': comment['textDisplay'],
                'Date': comment['updatedAt'] if 'updatedAt' in comment else comment['publishedAt']
            })

        next_page_token = reply_response.get('nextPageToken')
        if not next_page_token:
            break

    return replies

# Function to get all comments (including replies) for a single video
def get_comments_for_video(youtube, video_id):
    all_comments = []
    next_page_token = None

    while True:
        comment_request = youtube.commentThreads().list(
            part="snippet",
            videoId=video_id,
            pageToken=next_page_token,
            textFormat="plainText",
            maxResults=100
        )
        comment_response = comment_request.execute()

        for item in comment_response['items']:
            top_comment = item['snippet']['topLevelComment']['snippet']
            all_comments.append({
                'Timestamp': top_comment['publishedAt'],
                'Username': top_comment['authorDisplayName'],
                'VideoID': video_id,  # Directly using video_id from function parameter
                'Comment': top_comment['textDisplay'],
                'Date': top_comment['updatedAt'] if 'updatedAt' in top_comment else top_comment['publishedAt']
            })

            # Fetch replies if there are any
            if item['snippet']['totalReplyCount'] > 0:
                all_comments.extend(get_replies(youtube, item['snippet']['topLevelComment']['id'], video_id))

        next_page_token = comment_response.get('nextPageToken')
        if not next_page_token:
            break

    return all_comments

# List to hold all comments from all videos
all_comments = []


for video_id in video_ids:
    video_comments = get_comments_for_video(youtube, video_id)
    all_comments.extend(video_comments)

# Create DataFrame
comments_df = pd.DataFrame(all_comments)

In [7]:
# Function to get all comments (including replies) for a single video
def get_comments_for_video(youtube, video_id):
    all_comments = []
    next_page_token = None
    comment_count = 0 # Initialize comment count

    while True:
        comment_request = youtube.commentThreads().list(
            part="snippet",
            videoId=video_id,
            pageToken=next_page_token,
            textFormat="plainText",
            maxResults=100
        )
        comment_response = comment_request.execute()

        for item in comment_response['items']:
            if comment_count >= 100: # Check if comment count exceeds the limit
              break

            top_comment = item['snippet']['topLevelComment']['snippet']
            all_comments.append({
                'Timestamp': top_comment['publishedAt'],
                'Username': top_comment['authorDisplayName'],
                'VideoID': video_id,  # Directly using video_id from function parameter
                'Comment': top_comment['textDisplay'],
                'Date': top_comment['updatedAt'] if 'updatedAt' in top_comment else top_comment['publishedAt']
            })

            comment_count += 1 #Increment comment count

            # Fetch replies if there are any
            if item['snippet']['totalReplyCount'] > 0:
                replies = get_replies(youtube, item['snippet']['topLevelComment']['id'], video_id)
                reply_count = len(replies) # Get the number of replies
                if comment_count + reply_count <= 100: #Check if adding replies exceeds the limit
                  all_comments.extend(replies)
                  comment_count += reply_count # Increment comment count by number of replies


        next_page_token = comment_response.get('nextPageToken')
        if not next_page_token:
            break

    return all_comments
all_comments = []


for video_id in video_ids:
    video_comments = get_comments_for_video(youtube, video_id)
    all_comments.extend(video_comments)

# Create DataFrame
comments_df = pd.DataFrame(all_comments)

In [8]:
comments_df

,Timestamp,Username,VideoID,Comment,Date
0,2024-05-03T04:19:35Z,@kugastravel5180,yBltrN9-uaQ,My another Sunrise Express video⬇\nhttps://www...,2024-05-09T10:12:36Z
1,2024-09-18T13:00:23Z,@farmasinema,yBltrN9-uaQ,"Instead of highlight, I wish you also upload a...",2024-09-18T13:00:23Z
2,2024-09-17T21:33:48Z,@charlieshhh2030,yBltrN9-uaQ,Idk about that beef tongue bro,2024-09-17T21:33:48Z
3,2024-09-17T11:41:53Z,@mrspoonofbuttonmoon,yBltrN9-uaQ,Really enjoyed your humorous commentary 🤩,2024-09-17T11:41:53Z
4,2024-09-16T17:50:33Z,@austinnsuff6576,yBltrN9-uaQ,I like your travel vlogs because they are peac...,2024-09-16T17:50:33Z
...,...,...,...,...,...
95,2024-07-10T10:02:24Z,@Whippy99,yBltrN9-uaQ,Thank you for no sounds other than those that ...,2024-07-10T10:02:24Z
96,2024-07-09T05:35:52Z,@EDHBlvd,yBltrN9-uaQ,Super chill. 👌🏻,2024-07-09T05:35:52Z
97,2024-07-08T08:02:13Z,@scottyg9167,yBltrN9-uaQ,I find every one of your videos to be inexplic...,2024-07-08T08:02:13Z
98,2024-07-08T02:47:56Z,@highlyfavored4595,yBltrN9-uaQ,You got your snow,2024-07-08T02:47:56Z


### **Generating Feedback Using GEMINI**

In [9]:
simple_Text = comments_df.Comment.to_markdown()

In [ ]:
!pip install --upgrade google-cloud-aiplatform

In [11]:
from google.colab import auth
auth.authenticate_user()

In [12]:
import pathlib
import textwrap

import google.generativeai as genai

from IPython.display import display
from IPython.display import Markdown


def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

# Used to securely store your API key
from google.colab import userdata

In [13]:
GOOGLE_API_KEY=userdata.get('GeminiAPI')

genai.configure(api_key=GOOGLE_API_KEY)

In [14]:
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

models/gemini-1.0-pro-latest
models/gemini-1.0-pro
models/gemini-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-vision-latest
models/gemini-pro-vision
models/gemini-1.5-pro-latest
models/gemini-1.5-pro-001
models/gemini-1.5-pro
models/gemini-1.5-pro-exp-0801
models/gemini-1.5-pro-exp-0827
models/gemini-1.5-flash-latest
models/gemini-1.5-flash-001
models/gemini-1.5-flash-001-tuning
models/gemini-1.5-flash
models/gemini-1.5-flash-exp-0827
models/gemini-1.5-flash-8b-exp-0827


In [15]:
model = genai.GenerativeModel('gemini-flash')

In [16]:
prompt = f"""
               Role
              -------
              You are a youtuber who makes video on Data analytics.

              Task
              ------
              you just created a video on how to pull all the comments and replies from youtube videos using youtube API.

              Assess these comments for sentiment noting the percentage of positive negative and neutral comments.
              Please provide some example comments to back up your points on each section.

              Based on the analysis of the comments, what are the vewers demanding. Also analyze on what topic you should make next video?

              Comments
              ------
              {simple_Text}
"""



In [17]:
response = model.generate_content(prompt)

print(response.text)

**Sentiment Analysis**

**Positive:** 65
**Negative:** 12
**Neutral:** 16

**Positive Comments**

- "Really enjoyed your humorous commentary 🤩 The hot springs on a cold snowy day, excellent."
- "I like your travel vlogs because they are peaceful and quiet. I like just seeing the views and beauty and daydreaming about it instead of having a loud influencer yell at me."
- "The price was good, and the bed long enough (197 cm-I'm 185 cm tall), but narrow, and the ceiling low."
- "The noodle train! <3"
- "It was a very enjoyable trip from my view as well, a mere spectator. The snow was indeed lovely..."

**Negative Comments**

- "souless crap"
- "No black people and everything looks clean. Coincidence?"
- "They have to infest any showcase of Japan with their reductive  ethnonationalist horseshit."
- "Pure prejudice?"
- "Idk why this popped up in my feed but that was quite relaxing yet entertaining. Totally subscribing."

**Neutral Comments**

- "My another Sunrise Express video⬇"
- "Instead